## Ejemplo 2: Consultas a la base de datos y construcción de `DataFrames`

### 1. Objetivos:
    - Realizar consultas a la base de datos y construir un `DataFrame` para cada tabla.
 
---
    
### 2. Desarrollo:

#### a) Tablas a `DataFrames`

In [1]:
import pandas as pd
import sqlite3

Realizamos la conexión y obtenemos nuestro índice (cursor) para poder realizar operaciones con la base de datos:

In [2]:
bd = "movielens.sqlite3"
conn = sqlite3.connect(bd)
cur = conn.cursor()

cur

Ahora vamos a obtener la lista de todos los registros de la tabla `users` usando la función `execute(-sql-)` y la instrucción SQL:

`SELECT * FROM users`

In [3]:
sql = 'SELECT * FROM users'

Ahora hay que obtener todos los resultados de la consulta con:
    
`mi_cursor.fetchall()`

In [4]:
cursor = cur.execute(sql)

cursor

Si recuerdas el último ejemplo, `fetchall` nos regresa una `lista` de `tuplas`. Cada `tupla` representa una fila de nuestro conjunto de datos, por ejemplo obtén el primer registro se podría obtener como:

In [5]:
resultados = cursor.fetchall()
resultados

[(1, 'F', 1, 10, '48067'),
 (2, 'M', 56, 16, '70072'),
 (3, 'M', 25, 15, '55117'),
 (4, 'M', 45, 7, '02460'),
 (5, 'M', 25, 20, '55455'),
 (6, 'F', 50, 9, '55117'),
 (7, 'M', 35, 1, '06810'),
 (8, 'M', 25, 12, '11413'),
 (9, 'M', 25, 17, '61614'),
 (10, 'F', 35, 1, '95370'),
 (11, 'F', 25, 1, '04093'),
 (12, 'M', 25, 12, '32793'),
 (13, 'M', 45, 1, '93304'),
 (14, 'M', 35, 0, '60126'),
 (15, 'M', 25, 7, '22903'),
 (16, 'F', 35, 0, '20670'),
 (17, 'M', 50, 1, '95350'),
 (18, 'F', 18, 3, '95825'),
 (19, 'M', 1, 10, '48073'),
 (20, 'M', 25, 14, '55113'),
 (21, 'M', 18, 16, '99353'),
 (22, 'M', 18, 15, '53706'),
 (23, 'M', 35, 0, '90049'),
 (24, 'F', 25, 7, '10023'),
 (25, 'M', 18, 4, '01609'),
 (26, 'M', 25, 7, '23112'),
 (27, 'M', 25, 11, '19130'),
 (28, 'F', 25, 1, '14607'),
 (29, 'M', 35, 7, '33407'),
 (30, 'F', 35, 7, '19143'),
 (31, 'M', 56, 7, '06840'),
 (32, 'F', 25, 0, '19355'),
 (33, 'M', 45, 3, '55421'),
 (34, 'F', 18, 0, '02135'),
 (35, 'M', 45, 1, '02482'),
 (36, 'M', 25, 3, '

O podríamos obtener el género del primer usuario con:

In [6]:
resultados[0][1]

'F'

Algo muy util es obtener la lista de los nombres de las columnas de la tabla consultada, para eso se hacer uso del atributo:

`mi_cursor.description`

In [7]:
cursor.description

(('user_id', None, None, None, None, None, None),
 ('gender', None, None, None, None, None, None),
 ('age', None, None, None, None, None, None),
 ('occupation', None, None, None, None, None, None),
 ('cp', None, None, None, None, None, None))

y aplicando listas de compresión se puede obtener la lista deseada:

In [8]:
columnas = [x[0] for x in  cursor.description]
columnas


['user_id', 'gender', 'age', 'occupation', 'cp']

Afortunadamente `pandas` puede recibir justamente `listas` de `tuplas` como ingredientes para construir `DataFrames`. Sólo hace falta indicarle el nombre de las columnas. Los nombres de las columnas están especificados en el [archivo Readme.md](../../Datasets/MovieLens/Readme.md) que venía incluido con el dataset:

Así que ahora se crea el dataframe usando la forma:

`pd.DataFrame(-lista de registros-,
    columns=-lista de nombres de columnas-)`

In [9]:
df = pd.DataFrame(resultados, columns=columnas)

df.head()

user_id gender  age  occupation     cp
0        1      F    1          10  48067
1        2      M   56          16  70072
2        3      M   25          15  55117
3        4      M   45           7  02460
4        5      M   25          20  55455

Otra manera de realizar esta operación de una manera más compacta es utilizando la siguiente función de Pandas:

`pd.read_sql(-consulta sql-, -variable de conexión a la BD-)`

así que vamos a obtener justo el mismo dataframe y lu guardaremos en la viable `df_1`:

In [10]:
df_1 = pd.read_sql(sql, conn)

df_1.head()

user_id gender  age  occupation     cp
0        1      F    1          10  48067
1        2      M   56          16  70072
2        3      M   25          15  55117
3        4      M   45           7  02460
4        5      M   25          20  55455

Sería una buena idea convertir la columna `user_id` en índice para no tener información redundante usando:

`dataframe.set_index(-columna-, drop=True)`

y el resultado lo guardamos en la variable `df_2`:

In [11]:
df_2 = df_1.set_index('user_id', drop=True)

df_2.head(3)

gender  age  occupation     cp
user_id                               
1            F    1          10  48067
2            M   56          16  70072
3            M   25          15  55117

¡Listo! Vamos a guardar nuestros `DataFrames` en el archivo `users.csv` para que no haga falta volver a extraerlos de la base de datos. Ya podemos decirle adiós a nuestro fiel MySQL. ¡Chao, bellisimo!

In [12]:
df_2.to_csv("users.csv")

Ahora hagamos lo mismo para la tabla `occupations`, obtengamos todos los registros, examinemos, obtengamos los nombres de las columnas, creemos el dataframe, reasignemos índices en caso de ser necesario y guardemos el resultado en el archivo `occupations.csv`

In [13]:
df_3 = pd.read_sql("SELECT * FROM occupations", conn)
df_4 = df_3.set_index('occupation_id', drop=True)
#df_4.head()
df_4.to_csv("occupations.csv")

Haaa y no olvides cerrar tu índice y tu conexión:

In [14]:
cursor.close()
conn.close()

---
---

## Reto 2: Convertir tablas en MySQL a `DataFrames` de `pandas`

### 1. Objetivos:
    - Solicitar todos los datos de las tablas que están almacenadas en nuestra base de datos, convertirlos a `DataFrames` y guardarlos.
    
---
    
### 2. Desarrollo:

#### a) Tablas a `DataFrames`

Ya que tenemos nuestra conexión funcionando adecuadamente, vamos a utilizarla para realizar consultas a las base de datos y construir una base de datos local. Tu Reto consiste en los siguientes pasos:

1. Vuelve a establecer la conexión a la base de datos
2. Usando el comando `SELECT * FROM nombre_de_tabla`, realiza consultas a cada una de las 5 tablas que existen en la base de datos.
3. Crea un `DataFrame` por cada tabla que obtuviste. Para asignarle los nombres de las columnas correctamente, revisa el archivo [Readme.md](../../Datasets/MovieLens/Readme.md) donde está contenida toda esa información.
4. Asegúrate de que el índice sea adecuado en cada `DataFrame`. En los casos en los que haya datos redundantes, convierte una de las columnas en índice.
5. Si lo deseas ordena las columnas de la manera en la que mejor te parezca.
6. Guarda tus `DataFrames` en formato .csv para utilizarlos en los siguientes Retos.

> **NOTA IMPORTANTE**: La tabla movies es un poco complicada porque contiene muchos signos distintos. Tanto en la columna de nombre de película como la de género, encontramos signos como `,`, `:`, `.`, `|`. Esto hace un poco complicado el almacenamiento y lectura de este archivo. Si elijes guardar este archivo como un .csv separado por comas (`,`), a la hora de leerlo de regreso, `pandas` puede confundirse y pensar que el título de una película que contiene comas constituye dos columnas. Por esta razón, te recomiendo que la tabla `movies` la guardes agregando un separador poco convencional como `sep='$'`. De esta manera será muchísimo más fácil leer tu archivo de regreso usando ese separador.

In [15]:
bd = "movielens.sqlite3"
conn = sqlite3.connect(bd)

Compara con tus compañeros y revisa con la experta para que todos estén seguros de que tienen sus `DataFrames` estructurados de la manera correcta y que sus archivos .csv fueron creados exitosamente. Vamos a utilizar estos archivos en los Retos siguientes, así que es muy importante que tus datos estén estructurados adecuadamente.

In [16]:
sql = 'SELECT * FROM ratings'
df_ratings = pd.read_sql(sql, conn)
df_ratings = df_ratings.set_index(keys=["user_id", "movie_id"], drop=True)
df_ratings.to_csv("ratings.csv")

In [17]:
sql = 'SELECT * FROM age_ranges'
df_age_ranges = pd.read_sql(sql, conn)
df_age_ranges = df_age_ranges.set_index('age_id', drop=True)
df_age_ranges.to_csv("age_ranges.csv")

In [18]:
sql = 'SELECT * FROM movies'
df_movies = pd.read_sql(sql, conn)
df_movies = df_movies.set_index('movie_id', drop=True)
df_movies.to_csv("movies.csv", sep='#')

In [19]:
conn.close()